In [16]:
import datetime as dt
import pandas as pd
import numpy as np
import yfinance as yf
from scipy import optimize as sc
import seaborn as sns
import matplotlib.pyplot as plt

#plt.style.use('ggplot')
sns.set_style("darkgrid")
plt.rc("figure", figsize=(10, 8))
plt.rc("savefig", dpi=90)

def download(tickers: list, years: int):
    end_date = dt.datetime.now()
    start_date = end_date - dt.timedelta(days= 365 * years)
    mkt_data = yf.download(tickers, start= start_date, end= end_date, interval= '1d')['Adj Close']

    shares_outstanding = {}

    for ticker in tickers:
        end_date = dt.datetime.now()
        start_date = end_date - dt.timedelta(days= 365 * years)
        shares = yf.Ticker(ticker).get_shares_full(start= (dt.datetime.now() - dt.timedelta(days= 3)), end= None)
        shares_outstanding[ticker] = shares

    shares_outstanding = pd.DataFrame([shares_outstanding])
    return shares_outstanding, mkt_data

shares_outstanding, mkt_data = download(["SAMPO.HE", "ENENTO.HE", "KCR.HE", "GOFORE.HE", "NESTE.HE", "OMASP.HE", "QTCOM.HE", "REG1V.HE", "VALMT.HE", "ICP1V.HE", "METSO.HE", "KNEBV.HE", "NDA-FI.HE"], 1)

In [17]:
shares_outstanding

,SAMPO.HE,ENENTO.HE,KCR.HE,GOFORE.HE,NESTE.HE,OMASP.HE,QTCOM.HE,REG1V.HE,VALMT.HE,ICP1V.HE,METSO.HE,KNEBV.HE,NDA-FI.HE
0,2024-08-03 00:00:00+03:00 498312000 2024-08...,2024-08-03 00:00:00+03:00 23669300 2024-08-...,2024-08-03 00:00:00+03:00 79209104 2024-08-...,2024-08-03 00:00:00+03:00 15636500 2024-08-...,2024-08-03 00:00:00+03:00 768216000 2024-08...,2024-08-03 00:00:00+03:00 33138600 2024-08-...,2024-08-03 00:00:00+03:00 25391200 2024-08-...,2024-08-06 00:00:00+03:00 27251100 2024-08-...,2024-08-03 00:00:00+03:00 184168000 2024-08...,2024-08-03 00:00:00+03:00 29437200 2024-08-...,2024-08-06 00:00:00+03:00 827312000 2024-08...,2024-08-03 00:00:00+03:00 517527008 2024-08...,2024-08-06 00:00:00+03:00 3498990080 2024-0...


In [2]:
def market_capitalization(shares_outstanding):
    shares_outstanding = shares_outstanding.reindex(columns= mkt_data.columns[0:])
    shares_outstanding.to_dict()
    latest_price = mkt_data.iloc[-1].to_dict()
    shares_outstanding = {key: value[0] for key, value in shares_outstanding.items()}
    mkt_cap = {key: shares_outstanding[key] * latest_price[key] for key in shares_outstanding}
    total_mkt_cap = sum(mkt_cap.values())
    mkt_weights = {key: round(item / total_mkt_cap, 8) for key, item in mkt_cap.items()}
    mkt_weights = pd.DataFrame([mkt_weights])
    mkt_cap = pd.DataFrame([mkt_cap])

    return mkt_weights, mkt_cap

In [3]:
def annualized_portfolio_stdev(weights: np.ndarray, cov_matrix: pd.DataFrame, lambda_coeff):
    portfolio_stdev = np.sqrt(weights.T @ cov_matrix @ weights)*np.sqrt(252)
    l2_penalty = lambda_coeff * np.sum(weights**2)
    return portfolio_stdev + l2_penalty

def return_statistics(shares_outstanding, mkt_data, annual_rfr):
    mkt_weights, mkt_cap = market_capitalization(shares_outstanding)
    days = 365

    mkt_returns = np.log(mkt_data/mkt_data.shift(1)).dropna()
    daily_rfr = (1 + annual_rfr) ** (1 / days) - 1
    excess_mkt_returns = mkt_returns.subtract(daily_rfr, axis= 0)
    mkt_weights = mkt_weights.reindex(columns= excess_mkt_returns.columns[0:])
    
    weights_series = mkt_weights.loc[0]
    weights_array = weights_series.values

    #weighted_excess_returns = pd.DataFrame()
    #for row in excess_mkt_returns:
        #weighted_excess_returns[row] = excess_mkt_returns[row] * weights_series[row]
    #global_excess_return = weighted_excess_returns.mean()

    cov_matrix = excess_mkt_returns.cov() * 252
    #market_var = annualized_portfolio_stdev(weights_array, cov_matrix, 0)
    risk_aversion = 2.25

    return risk_aversion, cov_matrix, weights_array, weights_series, mkt_cap, excess_mkt_returns

risk_aversion, cov_matrix, weights_array, weights_series, mkt_cap, excess_mkt_returns = return_statistics(shares_outstanding, mkt_data, 0.03)

In [4]:
def implied_equilibrium_returns(risk_aversion: float, cov_matrix: pd.DataFrame, weights_array: np.ndarray):
    implied_equilibrium_returns = risk_aversion * cov_matrix.dot(weights_array).squeeze()
    return implied_equilibrium_returns

implied = implied_equilibrium_returns(risk_aversion, cov_matrix, weights_array)
implied

Ticker
ENENTO.HE    0.013549
GOFORE.HE    0.024616
ICP1V.HE     0.036833
KCR.HE       0.051071
KNEBV.HE     0.040960
METSO.HE     0.058485
NDA-FI.HE    0.044642
NESTE.HE     0.063311
OMASP.HE     0.036169
QTCOM.HE     0.052875
REG1V.HE     0.033647
SAMPO.HE     0.020205
VALMT.HE     0.043857
dtype: float64

In [5]:
pd.set_option('display.float_format', '{:.6f}'.format)

In [6]:
Q = np.array([0.05, 0.02, 0.01])
P = np.asarray([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0.98, 0, -0.02, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]])

In [7]:
def error_cov_matrix(cov_matrix, tau, P):
    matrix = np.diag(np.diag(P.dot(tau * cov_matrix).dot(P.T)))
    return matrix

tau = 0.025
omega = error_cov_matrix(cov_matrix, tau, P)

In [8]:
omega

array([[0.00182171, 0.        , 0.        ],
       [0.        , 0.00194604, 0.        ],
       [0.        , 0.        , 0.00087657]])

In [9]:
from numpy import linalg as lin

sigma_scaled = cov_matrix * tau
BL_return_vector = implied + sigma_scaled.dot(P.T).dot(lin.inv(P.dot(sigma_scaled).dot(P.T) + omega).dot(Q - P.dot(implied)))

In [10]:
returns_table = pd.concat([implied, BL_return_vector], axis=1) * 100
returns_table.columns = ['Implied Returns', 'BL Return Vector']
returns_table['Difference'] = returns_table['BL Return Vector'] - returns_table['Implied Returns']
returns_table.style.format('{:,.2f}%')

,Implied Returns,BL Return Vector,Difference
Ticker,,,
ENENTO.HE,1.35%,2.97%,1.62%
GOFORE.HE,2.46%,1.78%,-0.69%
ICP1V.HE,3.68%,2.93%,-0.75%
KCR.HE,5.11%,3.84%,-1.26%
KNEBV.HE,4.10%,3.38%,-0.72%
METSO.HE,5.85%,3.64%,-2.21%
NDA-FI.HE,4.46%,2.56%,-1.90%
NESTE.HE,6.33%,5.52%,-0.81%
OMASP.HE,3.62%,2.47%,-1.15%


In [11]:
inverse_cov = pd.DataFrame(lin.inv(cov_matrix.values), index=cov_matrix.columns, columns=cov_matrix.index)
BL_weights_vector = inverse_cov.dot(BL_return_vector)
BL_weights_vector = BL_weights_vector/sum(BL_weights_vector)

In [12]:
BL_weights_vector

Ticker
ENENTO.HE    0.150467
GOFORE.HE    0.003563
ICP1V.HE     0.003384
KCR.HE       0.048241
KNEBV.HE     0.252753
METSO.HE    -0.022088
NDA-FI.HE    0.139378
NESTE.HE     0.143696
OMASP.HE     0.004695
QTCOM.HE     0.020316
REG1V.HE     0.007739
SAMPO.HE     0.201032
VALMT.HE     0.046824
dtype: float64

In [13]:
# Calculate mean-variance optimised weights
MV_weights_vector = inverse_cov.dot(excess_mkt_returns.mean())
MV_weights_vector = MV_weights_vector/sum(MV_weights_vector)
weights_table = pd.concat([BL_weights_vector, weights_series, MV_weights_vector], axis=1) * 100
weights_table.columns = ['BL Weights', 'Market Cap Weights', 'Mean-Var Weights']
weights_table['BL/Mkt Cap Diff'] = weights_table['BL Weights'] - weights_table['Market Cap Weights']
weights_table.style.format('{:,.2f}%')

,BL Weights,Market Cap Weights,Mean-Var Weights,BL/Mkt Cap Diff
Ticker,,,,
ENENTO.HE,15.05%,0.35%,348.17%,14.69%
GOFORE.HE,0.36%,0.30%,-84.82%,0.05%
ICP1V.HE,0.34%,0.29%,-31.99%,0.05%
KCR.HE,4.82%,4.08%,-733.31%,0.74%
KNEBV.HE,25.28%,21.38%,-68.56%,3.89%
METSO.HE,-2.21%,6.26%,551.50%,-8.47%
NDA-FI.HE,13.94%,31.61%,-266.78%,-17.68%
NESTE.HE,14.37%,11.99%,304.67%,2.38%
OMASP.HE,0.47%,0.40%,480.02%,0.07%


In [17]:
import datetime as dt
import yfinance as yf
import pandas as pd
import numpy as np

end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days= 365 * 1)
data = yf.download('WRT1V.HE', start= start_date, end= end_date, interval= '1d')['Adj Close']

returns = np.log(data/data.shift(1)).dropna()
returns

[*********************100%%**********************]  1 of 1 completed


Date
2023-08-08    0.004055
2023-08-09    0.013399
2023-08-10    0.007073
2023-08-11   -0.011520
2023-08-14   -0.008503
                ...   
2024-07-31   -0.000525
2024-08-01   -0.018539
2024-08-02   -0.066304
2024-08-05   -0.019323
2024-08-06   -0.001749
Name: Adj Close, Length: 252, dtype: float64

In [24]:
(returns.mean()* 252 *  100 - 3) / (returns.std() * np.sqrt(252) * 100)

1.466185050910626

29.29775228612641

In [16]:
6.64131543103905 / 18.71

0.3549607392324452